In [2]:
%cd ../../
!pwd

/home/timur.bikbulatov/personal/aa_on_vad
/home/timur.bikbulatov/personal/aa_on_vad


/home/timur.bikbulatov/miniconda3/envs/aaml/lib/python3.11/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import torchaudio

In [4]:
import soundfile as sf
from pathlib import Path
import numpy as np
import torch
import IPython
from functools import partial
play = partial(IPython.display.Audio,
               rate=16000)
playr = IPython.display.Audio

In [67]:
import torch
from torch.utils.data import Dataset
import torchaudio
from pathlib import Path
import numpy as np
import soundfile as sr

class MusanMusicDataset(Dataset):
    def __init__(self, root_dir, segment_length:tuple[int,int]=(16000, 16000), target_sample_rate=16000, random_start_point:bool=True):
        """
        Args:
            root_dir (str): Directory with all the music wav files
            segment_length (int): Desired length of audio segments in samples
            sample_rate (int): Target sample rate
            transform (callable, optional): Optional transform to be applied on a sample
        """
        self.random_start_point = random_start_point
        self.root_dir = Path(root_dir)
        self.segment_length = segment_length
        self.target_sample_rate = target_sample_rate

        # Get all wav files recursively
        self.file_paths = list(self.root_dir.rglob("*.wav"))
        if not self.file_paths:
            raise RuntimeError(f"No .wav files found in {root_dir}")
        self.source_sample_rate = sr.read(self.file_paths[0])[1]
        self.resampler = torchaudio.transforms.Resample(self.source_sample_rate, self.target_sample_rate)
        # Cache file lengths to avoid repeated loading
        self.file_lengths = {}
        file_paths = []
        for path in self.file_paths:
            info = torchaudio.info(path)
            length = int(info.num_frames / self.source_sample_rate * self.target_sample_rate)
            if length >= self.segment_length[1]:
                file_paths.append(path)
                self.file_lengths[path] = length
        self.file_paths = file_paths

    
    def get_random_segment(self, idx, target_length):
        """
        Args:
            target_length (int): Desired length in samples
            
        Returns:
            torch.Tensor: Audio segment of specified length
        """
        # Shuffle file paths to randomize search
    
        file_path = self.file_paths[idx]
        file_length = self.file_lengths[file_path]
            
        # Load audio
        waveform, sr = torchaudio.load(file_path)
        if waveform.ndim > 1:
            print(waveform.shape)
            waveform = waveform[0, :]
        # Resample if necessary
        if sr != self.target_sample_rate:
            waveform = self.resampler(waveform)
        
        # Get random start point
        if self.random_start_point:
            max_start = file_length - target_length
            start_idx = np.random.randint(low=0,high=max_start, size=1)[-1]
        else:
            start_idx:int = 0    
        
        # Extract segment
        segment = waveform[start_idx:start_idx + target_length]
        return segment
        
        
    def __len__(self):
        return len(self.file_paths)
    
    def __getitem__(self, idx):
        """
        Get a random segment of the specified length from the dataset.
        
        Args:
            idx (int): Index (used for compatibility with DataLoader)
            
        Returns:
            torch.Tensor: Audio segment of length self.segment_length
        """
        segment_length = np.random.randint(low=self.segment_length[0],high=self.segment_length[1]+1, size=1)[-1]
        return self.get_random_segment(idx, segment_length)

    def check_length(self, idx):
        if self.file_lengths[self.file_paths[idx]] < self.segment_length[1]:
            return False
        return True



In [68]:
dataset = MusanMusicDataset(
    root_dir='datasets/musan/music',
    target_sample_rate=16000,
    segment_length=(16000, 32000)
    )

In [54]:
from types import MethodType
# dataset.get_random_segment = MethodType(get_random_segment, dataset)

In [59]:
dataset[1]

torch.Size([1, 3232496])
2165041 2181041
torch.Size([3232496])


tensor([0.2633, 0.3702, 0.4032,  ..., 0.3086, 0.2306, 0.4149])

In [72]:
w = dataset[1]
print(w.shape)
play(w)

torch.Size([1, 3232496])
torch.Size([29771])


In [19]:
len(dataset)

660

In [ ]:

# Example usage:
if __name__ == "__main__":
    # Initialize dataset
    dataset = MusanMusicDataset(
        root_dir="path/to/musan/music",
        segment_length=16000,  # 1 second at 16kHz
        sample_rate=16000
    )
    
    # Get a random segment
    audio_segment = dataset[0]  # Shape: [1, segment_length]
    
    # Get a segment of different length using the utility function
    longer_segment = dataset.get_random_segment(32000)  # 2 seconds
    
    # Use with DataLoader
    dataloader = torch.utils.data.DataLoader(
        dataset,
        batch_size=32,
        shuffle=True,
        num_workers=4
    )

In [24]:
np.random.randint(low=5, high=10, size=1)[0]

9